# 13. LARGE PL MODELS

1. [x] введение
2. [x] CoTexT
3. [x] упражнение
4. [x] ссылки

# 1. Введение

Отдельная модель под каждую задачу --- это хорошо. Тем не менее, иногда хочется иметь одну модель, которая умеет решать разные задачи:
- одну модель поддерживать проще, чем несколько (правда, если она не слишком большая)
- одна модель, обученная сразу под несколько задач, может справляться с задачами лучше, чем модели, обученные под каждую из задач в отдельности. Особенно это актуально, если задачи похожи

Multi-task learning --- обучение модели машинного обучения нескольким задачам одновременно. При этом, задачи не должны быть слишком похожими и не слишком различными. Как результат, возможно улучшение обобщающих способностей модели.

![](./res/13_multi-task.png)

# 2. CoTexT

[Phan et al - CoTexT: Multi-task Learning with Code-Text Transformer](https://aclanthology.org/2021.nlp4prog-1.5/) [@NLP4Prog 2021]

**Co**de and **Tex**t **T**ransfer Transformer)

- Transformer-based encoder-decoder
- NL-PL задачи:
    - суммаризация кода
    - генерация кода
    - детекция дефектов
    - исправление ошибок в коде
- языки: Java, Python, Javascript, PHP, Ruby etc

## 2.1 Модель

В основе лежит модель T5 --- **T**ext-**t**o-**T**ext **T**ransfer **T**ransformer ([Raffel et al., 2019](https://arxiv.org/abs/1910.10683)).

![](./res/13_t5.png)

## 2.2 Токенизация

Для токенизации используется [Sentence Piece Unsupervised Text Tokenizer](https://github.com/google/sentencepiece). В отличие от BPE пробелы тоже включаются в список символов, используемых в процессе токенизации. Однако специальные токены, используемые в коде (например, `[`, `{`, `$` и т. д.) не используются в модели Sentence Piece, но важный в языках программирования. Поэтому эти символы кодируются на естественном языке.
- фиксированный размер словаря
- не требует предварительного разбиения на части
- пробелы и специальный символы сохраняются, возможна детокенизация: `detokenized = ''.join(pieces).replace('▁', ' ')`
    - `<` -> `SMALLER_TOKEN`
    - `>` -> `GREATER_TOKEN`
    - `{` -> `OPEN_CURLY_TOKEN`
    - `}` -> `CLOSE_CURLY_TOKEN`
    - `[` -> `OPEN_SQUARE_TOKEN`
    - `]` -> `CLOSE_SQUARE_TOKEN`
    - `^` -> `EXPONENTIAL_TOKEN`
    - `#` -> `SHARP_TOKEN`
    - `$` -> `DOLLAR_TOKEN`    
    - `'` -> `UNK_TOKEN`    
    - `\n` -> `NEW_LINE_TOKEN`    
    - `\t` -> `INDENT`

## 2.3 Обучение

Данные:
- унимодальные данные (NL, PL)
- бимодальные данные (NL-PL)

Датасеты:
- [CodeSearchNet Corpus Collection](https://github.com/github/CodeSearchNet)
- [GitHub Repositories](https://console.cloud.google.com/marketplace/details/github/github-repos)

![](./res/13_cotext_pre-training.png)

Была использована модель [T5](https://huggingface.co/docs/transformers/model_doc/t5)-Base, обученная на [C4](https://commoncrawl.org/).
Все данные в текстовом формате (text-to-text). Тест и код конкатенируются в единую последовательность.
На этапе обучения в self-supervised режиме части последовательностей (span) случайно маскируются.

![](./res/13_cotext_fill-in-the-blank.png)

- 220M параметров
- lr = 0.001
- 1024 --- длина входной и выходной последовательностей
- TPU v2-8 on Google Colab
- batch_size = 128
- model parallelism = 2

#### CodeSearchNet Corpus Collection

- функции + описание на естественном языке
- GitHub
- Python, Java, Javascript, PHP, Ruby, Go

#### GitHub repositories

- функции + описание на естественном языке
- GitHub (Google BigQuery)
- Java, Python

### Multi-task

Обучение проходило единоообразно (максимальное правдоподобие) независимо от задач текст-код или код-текст. 
Для указания решаемой задачи использовался префикс (к входной последовательности).
Например, при файнтюнинге задачи CodeSummarization для каждого языка программирования добавляется префикс (язык программирования, например, Java) к входной последовательности.

![](./res/13_cotext_multi-task.png)

## 2.4 Evaluation

### CodeXGLUE

Использовался бенчмарк [CodeXGLUE](https://microsoft.github.io/CodeXGLUE/) (General Language Understanding Evaluation benchmark for CODE).
- набор задач анализа кода (10 задач, включая сценарии код-текст, текст-код, код-код и текст-текст)
- платформа для оценки модели
- таблица лидеров для сравнения

![](./res/13_cotext_datasets.png)

Оценивание происходило на задачах:
- Code Summarization
- Code Refinement

### Используемые длины последовательностей

![](./res/13_cotext_input_target.png)

### Code Summarization

Описать код на естественном языке. 
- CodeSearchNet
- Python, Java, Javascript, PHP, Ruby, Go

### Code Generation

Генерация кода по описанию
- CONCODE dataset
- Java

### Code Refinement (or Code Repair)

Исправление ошибок в коде.
- Bug2Fix corpus
- Java
- 50--100 токенов

### Defect Detection

Содержит ли код уязвимости или нет.
- Devign dataset
- C

### Метрики

- BLEU
- CodeBLEU

## 2.5 Результаты

Сравнение происходило с
- CodeGPT (GPT-2, CodeSearchNet dataset)
- CodeBERT (RoBERTa)
- PLBART (BART)

### Code Summarization

![](./res/13_cotext_code_summarization.png)

### Code Refinement

![](./res/13_cotext_code_refinement.png)

### Code Generation

![](./res/13_cotext_code_generation.png)

### Defect Prediction

![](./res/13_cotext_defect_prediction.png)

## 2.6 Реализация

- [модели](https://console.cloud.google.com/storage/browser/cotext)
- github: [justinphan3110/CoTexT](https://github.com/justinphan3110/CoTexT)
- huggingface: [razent/cotext-1-ccg](https://huggingface.co/razent/cotext-1-ccg)

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained('razent/cotext-1-ccg')
model = AutoModelForSeq2SeqLM.from_pretrained('razent/cotext-1-ccg')

In [ ]:
device_id = 'cpu'

sentence = 'def add(a, b): return a + b'
text = f'python: {sentence} </s>'

encoding = tokenizer.encode_plus(text, pad_to_max_length=True, return_tensors='pt')
input_ids, attention_masks = encoding['input_ids'].to(device_id), encoding['attention_mask'].to(device_id)

In [ ]:
outputs = model.generate(
    input_ids=input_ids, attention_mask=attention_masks,
    max_length=256,
    early_stopping=True
)

In [ ]:
for output in outputs:
    line = tokenizer.decode(output, skip_special_tokens=True, clean_up_tokenization_spaces=True)
    print(line)

# 3. Упражнение

Провести исследование применимости модели CoTexT для создания документации (Code Summarization).

# 4. Ссылки

- [Phan et al - CoTexT: Multi-task Learning with Code-Text Transformer](https://aclanthology.org/2021.nlp4prog-1.5/) 